In [1]:
import os
import math
import torch
import random
import shutil
import pickle
import pprint
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import models, transforms
from transformers import ViTFeatureExtractor, ViTModel, ViTImageProcessor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
warnings.filterwarnings("ignore")

/Users/xiaoyanli/opt/miniconda3/envs/cancer_immune/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Four models, Resnet152, Alexnet, VGG16, GoogleNet, ViT Classification

In [2]:
data_frame = pd.read_csv('oasis_cross-sectional.csv')
data_frame.drop('Delay', axis=1, inplace=True)
data_frame = data_frame.dropna()

with open('patients_codes_set.pkl', 'rb') as f:
    patients_codes_set = pickle.load(f)

print("Set loaded from patients_codes_set.pkl:", patients_codes_set)

Set loaded from patients_codes_set.pkl: {'0243', '0329', '0042', '0441', '0158', '0023', '0233', '0440', '0286', '0453', '0451', '0039', '0300', '0380', '0335', '0288', '0273', '0400', '0287', '0022', '0432', '0161', '0374', '0238', '0298', '0179', '0352', '0272', '0454', '0142', '0315', '0094', '0066', '0308', '0263', '0307', '0351', '0247', '0143', '0115', '0164', '0003', '0021', '0205', '0120', '0390', '0082', '0304', '0447', '0123', '0411', '0226', '0418', '0016', '0402', '0290', '0267', '0210'}


In [3]:
classification_dataframe = pd.DataFrame(columns=['Type', 'model', 'mode', 'Accuracy', 'Precision', 'Recall', 'True positive', 'Sensitivity', 'Specificity', 'F1 Score'])

lr = 1e-6
epochs = 100
num = 400
train_folder = './train_data'
# test_folder = './test_data'
test_folder = './train_data'

if not os.path.exists('./comparison_2'+'/models'):
    os.makedirs('./comparison_2'+'/models')

if not os.path.exists('./comparison_2'+'/results'):
    os.makedirs('./comparison_2'+'/results')

if not os.path.exists('./comparison_2'+'/test'):
    os.makedirs('./comparison_2'+'/test')

results_folder = './comparison_2'


##resnet152-Classification

In [4]:
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
resnet152 = models.resnet152(pretrained=True)
model_name = 'resnet152'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.resnet152 = models.resnet152(pretrained=True)
        # for param in self.resnet152.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        # self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.resnet152(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        # final_output = torch.cat((final_output, x), 1)
        # final_output = self.relu(final_output)
        # final_output = self.out_1(final_output)
        
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [5]:
folders = os.listdir(train_folder)
folders = [folder for folder in folders if os.path.isdir(train_folder + '/'+folder)]
print("Folders:", folders)

Folders: ['VeryMildDemented', 'ModerateDemented', 'MildDemented', 'NonDemented']


In [6]:
index = []
for i in range(len(folders)):
    folder = folders[i]
    if i == 0:
        y = 1
    elif i == 1:
        y = 3
    elif i == 2:
        y = 2
    else:
        y = 0
    for file in os.listdir(train_folder + '/' + folder):
        if file.endswith('.npy'):
            index.append([train_folder + '/' + folder + '/' + file, y])
# print(index)
# temp = []
# for ele in index:
#     if ele[0][-14:-10] in patients_codes_set:
#         temp.append([ele[0], ele[1]])
# index = temp

In [7]:
# resnet152 model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Epoch 0 loss: 176.95028483867645
Model saved to ./comparison_2/models/resnet152_MR1_1_class.pth


In [8]:
# adding resnet152 MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder + '/' + folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            data = data/255
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision   Recall  \
0  Classification  resnet152  MR1_1  0.267409    0.28141  1.05631   

   True positive  Sensitivity  Specificity  F1 Score  
0             96      0.28141     0.759956  0.444423  


In [9]:
# resnet152 model training with MR1_2

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 137.4839072227478
Model saved to ./comparison_2/models/resnet152_MR1_2_class.pth


In [10]:
# adding resnet152 MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_2  0.245125   0.259601  1.062692   

   True positive  Sensitivity  Specificity  F1 Score  
0             88     0.259601     0.753267  0.417268  


In [11]:
# resnet152 model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 123.77609729766846
Model saved to ./comparison_2/models/resnet152_MR1_3_class.pth


In [12]:
# adding resnet152 MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_3  0.269444   0.266165  0.986239   

   True positive  Sensitivity  Specificity  F1 Score  
0             97     0.266165     0.755215  0.419197  


In [13]:
# resnet152 model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 120.99772477149963
Model saved to ./comparison_2/models/resnet152_MR1_4_class.pth


In [14]:
# adding resnet152 MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_4  0.268608    0.26734  1.008157   

   True positive  Sensitivity  Specificity  F1 Score  
0             83      0.26734     0.756396  0.422613  


##ALexNet

In [15]:
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
alexnet = models.alexnet(pretrained=True)
model_name = 'alexnet'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.alexnet = models.alexnet(pretrained=True)
        # for param in self.alexnet.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.alexnet(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        # final_output = torch.cat((final_output, x), 1)
        # final_output = self.relu(final_output)
        # final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [16]:
# alexnet model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 176.16634809970856
Model saved to ./comparison_2/models/alexnet_MR1_1_class.pth


In [17]:
# adding alexnet MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision    Recall  \
0  Classification  alexnet  MR1_1  0.272981   0.245188  0.898441   

   True positive  Sensitivity  Specificity  F1 Score  
0             98     0.245188     0.748171  0.385242  


In [18]:
# alexnet model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 139.22005128860474
Model saved to ./comparison_2/models/alexnet_MR1_2_class.pth


In [19]:
# adding alexnet MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision    Recall  \
0  Classification  alexnet  MR1_2  0.264624   0.239742  0.910285   

   True positive  Sensitivity  Specificity  F1 Score  
0             95     0.239742     0.746521  0.379528  


In [20]:
# alexnet model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 119.1425529718399
Model saved to ./comparison_2/models/alexnet_MR1_3_class.pth


In [21]:
# adding alexnet MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision    Recall  \
0  Classification  alexnet  MR1_3  0.247222   0.243467  0.989332   

   True positive  Sensitivity  Specificity  F1 Score  
0             89     0.243467     0.747908  0.390769  


In [22]:
# alexnet model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 115.93560266494751
Model saved to ./comparison_2/models/alexnet_MR1_4_class.pth


In [23]:
# adding alexnet MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision    Recall  \
0  Classification  alexnet  MR1_4   0.28479   0.253399  0.889683   

   True positive  Sensitivity  Specificity  F1 Score  
0             88     0.253399       0.7513  0.394451  


## VGG16

In [24]:
# vgg16 model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model_name = 'vgg16'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.vgg16 = models.vgg16(pretrained=True)
        # for param in self.vgg16.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.vgg16(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        # final_output = torch.cat((final_output, x), 1)
        # final_output = self.relu(final_output)
        # final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [25]:
# vgg16 model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 178.17830914258957
Model saved to ./comparison_2/models/vgg16_MR1_1_class.pth


In [26]:
# adding vgg16 MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_1  0.278552   0.250853  0.900146            100   

   Sensitivity  Specificity  F1 Score  
0     0.250853     0.750234  0.392362  


In [27]:
# vgg16 model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 140.6702105998993
Model saved to ./comparison_2/models/vgg16_MR1_2_class.pth


In [28]:
# adding vgg16 MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_2  0.259053   0.234366  0.905936             93   

   Sensitivity  Specificity  F1 Score  
0     0.234366     0.744119  0.372394  


In [29]:
# vgg16 model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+ '/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 124.91962778568268
Model saved to ./comparison_2/models/vgg16_MR1_3_class.pth


In [30]:
# adding vgg16 MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_3  0.238889   0.231484  0.967078             86   

   Sensitivity  Specificity  F1 Score  
0     0.231484      0.74312  0.373553  


In [31]:
# vgg16 model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 120.25207942724228
Model saved to ./comparison_2/models/vgg16_MR1_4_class.pth


In [32]:
# adding vgg16 MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_4  0.291262   0.259497  0.890209             90   

   Sensitivity  Specificity  F1 Score  
0     0.259497     0.753577  0.401853  


GoogleNet

In [33]:
# googlenet model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model_name = 'googlenet'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.googlenet = models.googlenet(pretrained=True)
        # for param in self.googlenet.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.googlenet(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        # final_output = torch.cat((final_output, x), 1)
        # final_output = self.relu(final_output)
        # final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [34]:
# googlenet model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 175.33779442310333
Model saved to ./comparison_2/models/googlenet_MR1_1_class.pth


In [35]:
# adding googlenet MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_1  0.289694    0.26811  0.925466   

   True positive  Sensitivity  Specificity  F1 Score  
0            104      0.26811     0.756335   0.41577  


In [36]:
# googlenet model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 138.67900276184082
Model saved to ./comparison_2/models/googlenet_MR1_2_class.pth


In [37]:
# adding googlenet MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_2  0.267409   0.255736  0.954232   

   True positive  Sensitivity  Specificity  F1 Score  
0             96     0.255736     0.751168  0.403369  


In [38]:
# googlenet model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 121.09119784832001
Model saved to ./comparison_2/models/googlenet_MR1_3_class.pth


In [39]:
# adding googlenet MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_3  0.283333   0.277063  0.974286   

   True positive  Sensitivity  Specificity  F1 Score  
0            102     0.277063     0.758397  0.431437  


In [40]:
# googlenet model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 119.25980114936829
Model saved to ./comparison_2/models/googlenet_MR1_4_class.pth


In [41]:
# adding googlenet MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_4  0.242718   0.218321  0.906313   

   True positive  Sensitivity  Specificity  F1 Score  
0             75     0.218321     0.738452  0.351878  


Vision Transformers

In [42]:
# vision transformer model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model_name = 'vit'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        for param in self.vit_model.parameters():
            param.requires_grad = False
        # self.googlenet = models.googlenet(pretrained=True)
        # for param in self.googlenet.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(self.vit_model.config.hidden_size, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.vit_model(inputs['pixel_values'])
        

        outputs = outputs.last_hidden_state
        outputs = outputs.mean(dim=1)
        outputs = self.linear_layer1(outputs)


        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        # final_output = torch.cat((final_output, x), 1)
        # final_output = self.relu(final_output)
        # final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
epochs_vit = epochs*10

Device: cpu


In [43]:

# vit model training with MR1_1
lr = 1e-6

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 334.4242750406265
Epoch 100 loss: 271.60068559646606
Epoch 200 loss: 278.27223694324493
Epoch 300 loss: 304.51374995708466
Epoch 400 loss: 285.67174208164215
Epoch 500 loss: 245.10120391845703
Epoch 600 loss: 285.35123777389526
Epoch 700 loss: 281.14314925670624
Epoch 800 loss: 284.5495318174362
Epoch 900 loss: 281.3556822538376
Model saved to ./comparison_2/models/vit_MR1_1_class.pth


In [44]:
# adding vit MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision  Recall  True positive  \
0  Classification   vit  MR1_1  0.278552       0.25  0.8975            100   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.391068  


In [45]:
# vit model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 261.3802272081375
Epoch 100 loss: 277.16985642910004
Epoch 200 loss: 258.952943444252
Epoch 300 loss: 276.8822454214096
Epoch 400 loss: 294.44800996780396
Epoch 500 loss: 285.4630264043808
Epoch 600 loss: 279.3000407218933
Epoch 700 loss: 278.65793800354004
Epoch 800 loss: 300.38577151298523
Epoch 900 loss: 237.14545917510986
Model saved to ./comparison_2/models/vit_MR1_2_class.pth


In [46]:
# adding vit MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision  Recall  True positive  \
0  Classification   vit  MR1_2  0.278552       0.25  0.8975            100   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.391068  


In [47]:
# vit model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 265.97909104824066
Epoch 100 loss: 285.19770061969757
Epoch 200 loss: 300.1047258377075
Epoch 300 loss: 317.84916639328003
Epoch 400 loss: 298.2635762691498
Epoch 500 loss: 302.27912294864655
Epoch 600 loss: 295.19140625
Epoch 700 loss: 300.6781748533249
Epoch 800 loss: 293.4270610809326
Epoch 900 loss: 305.06348955631256
Model saved to ./comparison_2/models/vit_MR1_3_class.pth


In [48]:
# adding vit MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1_3  0.280556       0.25  0.891089            101   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.390456  


In [49]:
# vit model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 248.16565322875977
Epoch 100 loss: 270.7587819099426
Epoch 200 loss: 267.63083589076996
Epoch 300 loss: 207.87284290790558
Epoch 400 loss: 228.01447939872742
Epoch 500 loss: 273.79102075099945
Epoch 600 loss: 246.39422225952148
Epoch 700 loss: 244.8870130777359
Epoch 800 loss: 229.03336584568024
Epoch 900 loss: 281.6436047554016
Model saved to ./comparison_2/models/vit_MR1_4_class.pth


In [50]:
# adding vit MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1_4  0.281553       0.25  0.887931             87   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.390152  


In [53]:
classification_dataframe.to_csv(results_folder+'/test/classification_results.csv', index=False)

In [52]:
# MR1_1, MR1_2, MR1_3, MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        data = np.load(index[random_index][0])
        data = data/255
        label = index[random_index][1]
        code = index[random_index][0][-14:-10]
        x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
        x[:, 0] = (x[:, 0] - 18) / (96 - 18)
        x[:, 1] = (x[:, 1] - 5) / (5 - 1)
        x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
        x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
        x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
        if code not in patients_codes_set:
            x = np.zeros((1, 5))
        x = torch.tensor(x).float()
        y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
        model.train()
        optimizer.zero_grad()
        input_tensor = torch.clamp(torch.tensor(data), 0, 1)
        outputs = model(input_tensor, x)
        target = torch.tensor([label]).to(device)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv(results_folder+'/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = results_folder+'/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 1109.814453959465


KeyboardInterrupt: 

In [ ]:
# adding vit MR1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)